# Maîtriser l’utilisation des serveurs de calcul
## Lots de tâches
Le calcul haute-performance consiste non seulement au calcul parallèle par tâche, mais aussi au calcul de données en parallèle dans plusieurs tâches et/ou processus en simultané. Ce chapitre vous donnera les outils nécessaires pour gérer un grand nombre de tâches lorsque le projet de recherche requiert plusieurs centaines de résultats.

## GNU Parallel
### Cas d'utilisation
La commande `parallel` de [GNU Parallel](https://docs.computecanada.ca/wiki/GNU_Parallel/fr) permet d'utiliser pleinement les ressources locales d'un noeud de calcul, et ce, en gérant l'exécution d'une longue liste de tâches de *petite* taille.
C'est un peu comme l'ordonnanceur Slurm, mais à plus petite échelle et en gérant des processus au lieu de scripts de tâche.
Voici différents cas d'utilisation :
* Des centaines de tâches séquentielles, mais ...
  * un fichier de données à la fois
  * une combinaison de paramètres à la fois
  * une combinaison de paramètres et de fichiers de données à la fois
* Des dizaines de tâches parallèles à mémoire partagée (*multi-thread*) de quatre (4) processeurs

### Pourquoi pas Slurm?
OK, mais pourquoi ne pas tout simplement soumettre des centaines de tâches à Slurm?
* À tout moment, Slurm limite chaque usager à 1000 tâches en tout dans `squeue` (*pending* + *running*)
* Certains calculs sont tellement courts (< 5 minutes) que le démarrage et la fin de la tâche Slurm compteraient pour un pourcentage significatif du temps réel utilisé
  * On veut donc réduire les pertes de temps
* GNU Parallel nous évite d'utiliser un pipeline soumettant des centaines de scripts similaires, ce qui, dans bien des cas, facilite l'exécution de centaines de tâches de calcul semblables
* Enfin, si toutes vos tâches envoient des notifications par courriel, imaginez recevoir ces centaines de courriels!

Les autres avantages à considérer :
* Par défaut, le nombre de processeurs disponibles limite le nombre de tâches de calcul exécutées en simultané
  * Dans le cas de calculs parallèles, c'est possible de limiter davantage le nombre de processus
* GNU Parallel peut [reprendre la séquence des tâches de calcul](https://docs.computecanada.ca/wiki/GNU_Parallel/fr#Suivi_des_commandes_ex.C3.A9cut.C3.A9es_ou_des_commandes_ayant_.C3.A9chou.C3.A9.3B_fonctionnalit.C3.A9s_de_red.C3.A9marrage) en cas de fin hâtive de la tâche Slurm

### Syntaxe de la commande GNU Parallel
Les éléments de base de la commande `parallel` :
* Options de configuration de la commande `parallel` :
  * Si un fichier de paramètres est fourni, quel serait le séparateur (`--colsep`) et s'il y a une entête (`--header`)
* Le gabarit de la commande à exécuter :
  * Le ou les paramètres changeants sont donnés via des paires `{}`
  * Lorsqu'il y a plusieurs paramètres dans le gabarit, c'est possible de les numéroter. Par exemple : `{1}`, `{2}`, etc.
* Après le gabarit de commande, les différentes valeurs de chaque paramètre sont spécifiées :
  * sur la même ligne de commande après le séparateur `:::`
  * dans un fichier texte dont le nom est donnée après le séparateur `::::`

Voici [deux exemples tirés de la documentation](https://docs.computecanada.ca/wiki/GNU_Parallel/fr#Sp.C3.A9cifier_plusieurs_arguments) :
```
[name@server ~]$ parallel echo {2} {1} ::: 7 8 ::: $(seq 1 3)
1 7
2 7
1 8
3 7
2 8
3 8
```
```
[name@server ~]$ cat param.tsv
Param_A Param_B Param_C
7       8       9
10      11      12
[name@server ~]$ parallel --colsep='\t' --header=1 echo a={3} b={1} :::: param.tsv
a=9 b=7
a=12 b=10
```

### Exercice - Paramètres de Benchmark5D
Dans cet exercice, il est question d'une recherche de paramètres optimaux pour un code "quelconque". En fait, le code utilise $O(n^3)$ valeurs pour $O(n^5)$ opérations à double précision. En vous épargnant les détails du programme, les différents paramètres à tester permettent de démontrer les effets de la mémoire cache (L1, L2 et L3). Ainsi, de petites valeurs de `p*` et/ou `b*` forcent le programme à calculer le plus localement possible dans la mémoire-vive.

Pour installer le programme Benchmark5D :
* `bash scripts/installer-b5D.sh`

Pour lancer une tâche de 4 processeurs utilisant GNU Parallel pour tester 3*4 combinaisons de paramètres :
* `sbatch scripts/executer-b5D.sh`

Pour le suivi de la tâche :
* `squeue -u $USER`
* Les résultats seront dans un fichier `slurm-<jobid>.out`
  * `cat slurm-<jobid>.out`
* À la fin de la tâche, on peut analyser les ressources utilisées :
  * `seff <jobid>`

## Vecteurs de tâches
Alors que GNU Parallel s'utilise bien avec des calculs séquentiels et parallèles de petite taille (<32 processeurs),
GNU Parallel n'est pas vraiment l'outil approprié pour un lot de longs calculs parallèles de plus grande taille (>=32 processeurs par calcul).
Dans le cas où un même programme parallèle doit être exécuté avec différentes combinaisons de paramètres,
il y a moyen de soumettre un seul [vecteur de tâches](https://docs.computecanada.ca/wiki/Job_arrays/fr) et de coder le script de tâche de telle sorte que les paramètres seront déterminés en fonction d'un indice unique du vecteur de tâches.

Pour soumettre un vecteur de tâches à l'ordonnanceur Slurm, à la ligne de commande `sbatch` ou dans l'entête `#SBATCH` du script de tâche, on doit ajouter l'option `--array=<indices>`.
Voir [ici quelques exemples](https://docs.computecanada.ca/wiki/Job_arrays/fr).

L'identifiant d'une tâche Slurm dans un vecteur de tâches contient :
* L'identifiant du vecteur de tâches
* Caractère de soulignement (`_`)
* L'indice unique associé à la tâche

Par exemple : `25249551_15`

Dans le script de tâche, la variable d'environnement `$SLURM_ARRAY_TASK_ID` peut être utilisée pour retrouver la valeur actuelle de l'indice unique associé à la tâche en cours d'exécution, soit un entier parmis les `<indices>`.
La variable `$SLURM_ARRAY_TASK_ID` peut être utilisée de différentes façons :
* `nom_fichier.$SLURM_ARRAY_TASK_ID`
* `nom_dossier-$SLURM_ARRAY_TASK_ID`
* `PARAM=$SLURM_ARRAY_TASK_ID`
* `PARAM1=$((SLURM_ARRAY_TASK_ID / 12))`
* `PARAM2=$((SLURM_ARRAY_TASK_ID % 12))`
* `PARAMS=$(sed -n "${SLURM_ARRAY_TASK_ID}p" combinaisons.txt)`

### Exercice - Vecteur de tâches
Pour lancer le vecteur de tâches :
* `sbatch scripts/executer-sleep.sh`

Après l'exécution des trois tâches, on peut inspecter les résultats :
* `cat slurm-*_*.out`

## Le *Greedy Launcher Of Small Tasks*, ou GLOST